## Data Wrangling: Combining Data

This notebook follows the work completed in the Data Wrangling: Web Scraping notebook. Now that the raw files have been created and stored, the data will be further consolidated into their listings and reviews categories in the form of the pandas dataframe.

In [1]:
# import relevant packages
import pandas as pd
import shutil
import os
import time
from datetime import datetime

#### Explore a single dataframe
Since I'm working with so much, I want to incorporate elements into my data cleaning functions to also minimize the files size. It will help to know the datatypes in the data, so I will explore that by looking at two of the smallest files that I scraped from the web.

In [127]:
# explore one dataframe for listings and reviwes data to explore datatypes
listing_file = '/Users/limesncoconuts2/springboard_data/data_capstone_one/web_scraped/salem-or_2018-12-08_listings.csv.gz'
review_file = '/Users/limesncoconuts2/springboard_data/data_capstone_one/web_scraped/salem-or_2018-09-11_reviews.csv.gz'
listing = pd.read_csv(listing_file)
review = pd.read_csv(review_file)

##### Listings #####
First let's look at listings data:

In [6]:
# check the memory
listing.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 96 columns):
id                                  192 non-null int64
listing_url                         192 non-null object
scrape_id                           192 non-null int64
last_scraped                        192 non-null object
name                                192 non-null object
summary                             190 non-null object
space                               128 non-null object
description                         191 non-null object
experiences_offered                 192 non-null object
neighborhood_overview               128 non-null object
notes                               108 non-null object
transit                             108 non-null object
access                              128 non-null object
interaction                         129 non-null object
house_rules                         118 non-null object
thumbnail_url                       0 non-null float64
me

In [14]:
# show all columns
pd.set_option('display.max_columns', None)

In [15]:
listing.head(1)

id                         listing_url       scrape_id last_scraped  \
0  41974  https://www.airbnb.com/rooms/41974  20181208175127   2018-12-08   

                name                                            summary  \
0  Englewood Cottage  Enjoy a peaceful stay in part of a 1938 cottag...   

                                               space  \
0  These are cozy and artful private rooms: a bed...   

                                         description experiences_offered  \
0  Enjoy a peaceful stay in part of a 1938 cottag...                none   

                               neighborhood_overview  \
0  Englewood is an old, charming and diverse, cen...   

                                               notes  \
0  There are so many things to appreciate in this...   

                                             transit  \
0  The Cherriots bus stop is 3 blocks away, the n...   

                                              access  \
0  This is a house with two sections. You will ha...   

                                         interaction  \
0  Your host, and occassionaly her 5 year old gra...   

                                         house_rules  thumbnail_url  \
0  quiet time between 10pm and 8am no smoking any...            NaN   

   medium_url                                        picture_url  \
0         NaN  https://a0.muscache.com/im/pictures/23573640/d...   

   xl_picture_url  host_id                                  host_url  \
0             NaN   183514  https://www.airbnb.com/users/show/183514   

  host_name  host_since                 host_location  \
0      Ruth  2010-07-28  Salem, Oregon, United States   

                                          host_about  host_response_time  \
0  I am an actively engaged, retired restaurant o...  within a few hours   

  host_response_rate  host_acceptance_rate host_is_superhost  \
0               100%                   NaN                 t   

                                  host_thumbnail_url  \
0  https://a0.muscache.com/im/users/183514/profil...   

                                    host_picture_url  host_neighbourhood  \
0  https://a0.muscache.com/im/users/183514/profil...                 NaN   

   host_listings_count  host_total_listings_count  \
0                    1                          1   

                                 host_verifications host_has_profile_pic  \
0  ['email', 'phone', 'facebook', 'reviews', 'kba']                    t   

  host_identity_verified                    street  neighbourhood  \
0                      t  Salem, OR, United States            NaN   

  neighbourhood_cleansed  neighbourhood_group_cleansed   city state  zipcode  \
0                 Ward 1                           NaN  Salem    OR  97301.0   

             market smart_location country_code        country   latitude  \
0  Other (Domestic)      Salem, OR           US  United States  44.944043   

   longitude is_location_exact property_type        room_type  accommodates  \
0 -123.00996                 t     Apartment  Entire home/apt             4   

   bathrooms  bedrooms  beds  bed_type  \
0        1.0       1.0   2.0  Real Bed   

                                           amenities  square_feet   price  \
0  {TV,Wifi,"Air conditioning",Kitchen,Breakfast,...          NaN  $80.00   

  weekly_price monthly_price security_deposit cleaning_fee  guests_included  \
0      $345.00       $900.00              NaN          NaN                2   

  extra_people  minimum_nights  maximum_nights calendar_updated  \
0       $20.00               1              30      6 weeks ago   

  has_availability  availability_30  availability_60  availability_90  \
0                t                0                0                0   

   availability_365 calendar_last_scraped  number_of_reviews first_review  \
0               112            2018-12-08                151   2010-10-18   

  last_review  review_scores_rating  review_scores_accuracy  \
0  2018-

In order to check if prices were in US dollars across countries, I checked data from Beijing, China amd Bordeaux, France. It looks like prices seem to be all in dollars!

In [43]:
beijing = pd.read_csv('/Users/limesncoconuts2/springboard_data/data_capstone_one/web_scraped/beijing_2018-04-16_listings.csv.gz')
beijing[['price', 'weekly_price','monthly_price']].head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (43,61,62,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


price weekly_price monthly_price
0    $740.00    $7,423.00    $24,472.00
1  $1,199.00    $7,200.00    $28,800.00
2    $609.00    $4,760.00           NaN
3    $615.00    $3,710.00           NaN
4    $665.00    $4,200.00           NaN

In [44]:
bordeaux = pd.read_csv('/Users/limesncoconuts2/springboard_data/data_capstone_one/web_scraped/bordeaux_2018-06-20_listings.csv.gz')
bordeaux[['price', 'weekly_price','monthly_price']].head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (43,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


price weekly_price monthly_price
0   $24.00      $200.00       $500.00
1   $71.00      $350.00     $1,250.00
2  $100.00          NaN           NaN
3   $75.00          NaN           NaN
4  $100.00      $630.00           NaN

It turns out that a lot of columns can be converted to more accurate, and oftentimes, space-saving datatypes. Additionally, the prices and percentages can be changed into float numbers after removing the '$' and '%' signs from the strings.

In [110]:
to_int = ['scrape_id','zipcode','bedrooms','beds','host_listings_count','host_total_listings_count']
to_datetime = ['last_scraped','host_since','calendar_last_scraped', 'first_review', 'last_review']
to_bool = ['requires_license','host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','instant_bookable','is_business_travel_ready','require_guest_profile_picture','require_guest_phone_verification']
to_category = ['property_type','room_type','bed_type','cancellation_policy','market','neighbourhood','city','state','calendar_updated','host_neighbourhood']

prices = ['price','weekly_price','monthly_price','security_deposit','cleaning_fee','extra_people']
percentages = ['host_response_rate', 'host_acceptance_rate']

Let's test the datatype conversion on the listing dataframe:

In [133]:
listing = pd.read_csv(listing_file)

for col_name in to_int:
    listing[col_name] = pd.to_numeric(listing[col_name], downcast='integer', errors='coerce')

# dates are all in YYYY-MM-DD format
for col_name in to_datetime:
    listing[col_name] = pd.to_datetime(listing[col_name], format='%Y-%m-%d')

# booleans start out as 't' or 'f' strings
for col_name in to_bool:
    listing[col_name] = listing[col_name] == 't'
    
for col_name in to_category:
    listing[col_name] = listing[col_name].astype('category')
    
## change price columns
for col_name in prices:
    listing[col_name] = pd.to_numeric(listing[col_name].str.replace('$',''), downcast='float', errors='coerce')
    # rename column
    listing.rename(columns={col_name: col_name + '_USD'}, inplace=True)

## change percentage columns
for col_name in percentages:
    try:
        listing[col_name] = pd.to_numeric(listing[col_name].str.replace('%',''), downcast='float', errors='coerce')
    except AttributeError:
        pass
    # rename column
    listing.rename(columns={col_name: col_name + '_percentage'}, inplace=True)

In [141]:
listing[['price_USD', 'host_response_rate_percentage']]

price_USD  host_response_rate_percentage
0         80.0                          100.0
1         75.0                          100.0
2         35.0                          100.0
3        105.0                          100.0
4        175.0                            NaN
5         50.0                          100.0
6         65.0                          100.0
7         50.0                          100.0
8         55.0                          100.0
9         67.0                          100.0
10        54.0                          100.0
11        95.0                          100.0
12       220.0                          100.0
13        75.0                          100.0
14       106.0                          100.0
15        38.0                           90.0
16       125.0                            NaN
17        85.0                          100.0
18        85.0                          100.0
19        79.0                          100.0
20       115.0                          100.0
21        39.0                          100.0
22        40.0                          100.0
23        58.0                          100.0
24        30.0                          100.0
25        95.0                          100.0
26       120.0                          100.0
27        50.0                            NaN
28        38.0                          100.0
29        46.0                            NaN
..         ...                            ...
162      140.0                          100.0
163       22.0                          100.0
164       85.0                          100.0
165       98.0                          100.0
166       18.0                          100.0
167      310.0                          100.0
168       50.0                          100.0
169       30.0                           40.0
170       45.0                          100.0
171       60.0                          100.0
172       38.0                          100.0
173       50.0                          100.0
174      195.0                           76.0
175       80.0                          100.0
176       80.0                          100.0
177       25.0                          100.0
178       80.0                          100.0
179      140.0                          100.0
180       59.0                          100.0
181      120.0                          100.0
182       35.0                          100.0
183      110.0                          100.0
184       45.0                          100.0
185      150.0                            NaN
186      125.0                            NaN
187       38.0                          100.0
188       54.0                          100.0
189       40.0                          100.0
190       19.0                            0.0
191      500.0                            NaN

[192 rows x 2 columns]

In [91]:
listing.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 96 columns):
id                                  192 non-null int64
listing_url                         192 non-null object
scrape_id                           192 non-null int64
last_scraped                        192 non-null datetime64[ns]
name                                192 non-null object
summary                             190 non-null object
space                               128 non-null object
description                         191 non-null object
experiences_offered                 192 non-null object
neighborhood_overview               128 non-null object
notes                               108 non-null object
transit                             108 non-null object
access                              128 non-null object
interaction                         129 non-null object
house_rules                         118 non-null object
thumbnail_url                       0 non-null fl

By converting datatypes, we saved 0.3 MB - not a ton, but at least the we are making our storage of the data more efficient!

##### Reviews #####
Now let's look at reviews data:

In [45]:
# check the memory
review.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5995 entries, 0 to 5994
Data columns (total 6 columns):
listing_id       5995 non-null int64
id               5995 non-null int64
date             5995 non-null object
reviewer_id      5995 non-null int64
reviewer_name    5995 non-null object
comments         5991 non-null object
dtypes: int64(3), object(3)
memory usage: 2.8 MB


In [46]:
review.head(1)

listing_id      id        date  reviewer_id reviewer_name  \
0       41974  121517  2010-10-18       143866          Inge   

                                            comments  
0  I was very comfortable staying with Ruth. Her ...

It looks like for reviews data, the only type coversion that could happen would be to change the 'date' column from object to datetime.

In [66]:
review['date'] = pd.to_datetime(review['date'], format='%Y-%m-%d')

In [67]:
review.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5995 entries, 0 to 5994
Data columns (total 6 columns):
listing_id       5995 non-null int64
id               5995 non-null int64
date             5995 non-null datetime64[ns]
reviewer_id      5995 non-null int64
reviewer_name    5995 non-null object
comments         5991 non-null object
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 2.5 MB


Just from converting the dates from strings to datetime type, we saved 0.3 MB - not bad!

Now we can implement what we learned into some functions to make data processing easier.

### Functions to Consolidate the Data
Because there are hundreds of raw files to process with hundreds of thousands of lines of data, it helps to create functions that will do the heavy lifting for us. This heavy lifing includes:
1. Checking if the consolidated csv files we want already exist on the computer: **consolidate_data**.
2. Concatenating data of the same city and category (listings or reviews) together: **combine_listings, combine_reviews, and concat_files**.
3. Changing data types to optimize storage efficiency: **change_datatypes**.
4. Saving the concatenated data as a csv file: **export_csv**.

In [2]:
def consolidate_data(city, directory, target):
    """ Checks if the csv file for either listings
        or reviews data has been created for the designated
        city in the target folder.
        If the file has not been created, run the combine_listings
        or combine_reviews function for that city, and then create
        the csv file for that city.
    """
    
    filename = city + '_listings.csv'
    # if listings file for this city doesn't already exist, create listings_df and save as csv
    if(not os.path.isfile(target + filename)):
        listings_df = combine_listings(city, directory)
        export_csv(city, filename, listings_df, target)
    
    filename = city + '_reviews.csv'
    # if reviews file for this city doesn't already exist, create reviews_df and save as csv
    if(not os.path.isfile(target + filename)):
        reviews_df = combine_reviews(city, directory)
        export_csv(city, filename, reviews_df, target)



In [3]:
#### FUNCTION FOR LISTINGS #### 
def combine_listings(city, directory):
    """ Goes through files in the directory and checks for the
        designated city listings files. Appends the names of the
        listings files of that city to a list, and passes the list
        and the directory name to the concat_files function.
    """
    
    target_files = []
    
    for file in os.listdir(directory):
        # check if file from the target city and is listings data
        if city in file and 'listings' in file:
            # add to list of target files
            target_files.append(file)
            
    # concatenate files in list
    return concat_files(target_files, directory, 'listing') 

In [4]:
#### FUNCTION FOR REVIEWS #### 
def combine_reviews(city, directory):
    """ Goes through files in the directory and checks for the
        designated city reviews files. Add the names of the
        reviews files of that city to a list, and passes the list
        and the directory name to the concat_files function.
    """
    target_files = []
    
    for file in os.listdir(directory):
        # check if file from the target city and is listings data
        if city in file and 'reviews' in file:
            # add to list of target files
            target_files.append(file)
            
    # concatenate files in list
    return concat_files(target_files, directory, 'review') 

In [5]:
def concat_files(file_list, directory, kind):
    """Creates a pandas dataframe for each file name in the 
       list of files, then adds the date recorded as a column
       in that dataframe (taken from the file name). Appends
       the dataframe to a list of dataframes. After all files
       in the list have been converted to pandas dataframes,
       concatenate the dataframes together, drop duplicates (ignoring the date_recorded column),
       and reset the dataframe index.
    """
    ### ADD THINGS TO MAKE DATAFRAMES MORE EFFICIENT ###
    # change datatypes to be more efficient
    all_dfs = []
    
    for file in file_list:
        # make into a pandas dataframe
        df = pd.read_csv(directory + file)
        
        # add column of the date
        df['date_recorded'] = file.split('_')[1]
        
        # get rid of duplicates, ignoring new date column
        df = df.drop_duplicates(df.columns.difference(['date_recorded']))
        
        # change datatypes
        df = change_datatypes(df, kind)
        
        # append to a list of dataframes
        all_dfs.append(df)
    
    # append dataframes together along x-axis
    concat_all = pd.concat(all_dfs)

    # reset index
    concat_all.reset_index(drop=True, inplace=True)
    return concat_all

In [6]:
def change_datatypes(df, kind):
    """ Changes the datatypes of specified columns in the dataframe,
        depending on if the dataframe passed to the function has
        listings data or reviews data. Has try/except blocks just in case
        a certain column is not found in a dataframe.
    """
    # lists of columns that need to be converted in listings data
    to_int = ['scrape_id','zipcode','bedrooms','beds','host_listings_count','host_total_listings_count']
    to_datetime = ['last_scraped','host_since','calendar_last_scraped', 'first_review', 'last_review']
    to_bool = ['requires_license','host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','instant_bookable','is_business_travel_ready','require_guest_profile_picture','require_guest_phone_verification']
    to_category = ['property_type','room_type','bed_type','cancellation_policy','market','neighbourhood','city','state','calendar_updated','host_neighbourhood']
    prices = ['price','weekly_price','monthly_price','security_deposit','cleaning_fee','extra_people']
    percentages = ['host_response_rate', 'host_acceptance_rate']
    
    if kind == 'listing':
        ## to integer
        for col_name in to_int:
            try:
                df[col_name] = pd.to_numeric(df[col_name], downcast='integer', errors='coerce')
            except KeyError:
                pass

        ## to datetime: dates are all in YYYY-MM-DD format
        for col_name in to_datetime:
            try:
                df[col_name] = pd.to_datetime(df[col_name], format='%Y-%m-%d')
            except KeyError:
                pass

        ## to bool: booleans start out as 't' or 'f' strings
        for col_name in to_bool:
            try:
                df[col_name] = df[col_name] == 't'
            except:
                pass
    
        ## to category
        for col_name in to_category:
            try:
                df[col_name] = df[col_name].astype('category')
            except KeyError:
                pass
       
        ## change price columns
        for col_name in prices:
            try:
                df[col_name] = pd.to_numeric(df[col_name].str.replace('$',''), downcast='float', errors='coerce')
            except:
                pass
            # rename column
            df.rename(columns={col_name: col_name + '_USD'}, inplace=True)

        ## change percentage columns
        for col_name in percentages:
            try:
                df[col_name] = pd.to_numeric(df[col_name].str.replace('%',''), downcast='float', errors='coerce')
            except:
                pass
            # rename column
            df.rename(columns={col_name: col_name + '_percentage'}, inplace=True)
    
    else:
        ## to datetime
        try:
            df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
        except KeyError:
            pass
    
    return df

In [7]:
def export_csv(city, filename, df, target):
    """ If the desired csv file does not exist in the current
        working directory, convert the dataframe to a csv file
        and move the the desired folder in the target directory.
    """
    current_dir = os.getcwd() + '/' + filename
    # export listings dataframe to csv if file doesn't already exist
    if(not os.path.isfile(current_dir)):
        df.to_csv(filename, index=False)
        # move csv to target directory
        shutil.move(current_dir, target)

#### Debugging and Time Test ####
Before I run the functions on all of the data, I will run a time test on just one city: San Diego.

In [157]:
directory = '/Users/limesncoconuts2/springboard_data/data_capstone_one/web_scraped/'
target = '/Users/limesncoconuts2/springboard_data/data_capstone_one/csv/'
city = 'san-diego'

start_time = time.time() # timestamp
if(not os.path.isfile(target + city + '_listings.csv') or not os.path.isfile(target + city + '_reviews.csv')):
        consolidate_data(city, directory, target)

time_to_run = (time.time() - start_time)/60 # timestamp, calculate function time
print('Time:',time_to_run, 'minutes')

# delete the files that were created
os.remove(target + city + '_listings.csv')
os.remove(target + city + '_reviews.csv')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys
/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Time: 1.870709979534149 minutes


From testing on one city, I found that not all listings data have the 'is_business_ready' column. Collection of this data must have something that must have begun after Airbnb started their special business travel options in 2017. Therefore, files made before that year do not have the 'is_business_ready' column. I added a try/except block around the check against the boolean columns to pass this error.

Once it worked, getting the data for San Diego took under two minutes, so I don't expect the creation of the .csv files for all 84 cities to take more than 168 minutes, or just about 2 and two-thirds hours.

In [97]:
ls = ['san-diego_2018-07-16_listings.csv.gz', 'san-diego_2018-04-14_listings.csv.gz', 'san-diego_2018-08-16_listings.csv.gz', 'san-diego_2018-09-13_listings.csv.gz', 'san-diego_2018-10-11_listings.csv.gz', 'san-diego_2018-11-15_listings.csv.gz','san-diego_2016-07-07_listings.csv.gz', 'san-diego_2015-06-22_listings.csv.gz']
for i in ls:
    print(i)
    sd = pd.read_csv('/Users/limesncoconuts2/springboard_data/data_capstone_one/web_scraped/' + i)
    try:
        sd['is_business_travel_ready'] == 't'
    except KeyError:
        print("No 'is_business_ready' column")

san-diego_2018-07-16_listings.csv.gz


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


san-diego_2018-04-14_listings.csv.gz
san-diego_2018-08-16_listings.csv.gz
san-diego_2018-09-13_listings.csv.gz
san-diego_2018-10-11_listings.csv.gz
san-diego_2018-11-15_listings.csv.gz
san-diego_2016-07-07_listings.csv.gz
No 'is_business_ready' column
san-diego_2015-06-22_listings.csv.gz
No 'is_business_ready' column


#### The following code uses the above functions on the project data:

In [8]:
# identify the directory and target folder
directory = '/Users/limesncoconuts2/springboard_data/data_capstone_one/web_scraped/'
target = '/Users/limesncoconuts2/springboard_data/data_capstone_one/csv/'

In [9]:
# get list of unique cities in alphabetical order
unique_cities = []
for file in os.listdir(directory):
    unique_cities.append(file.split('_')[0])
unique_cities = list(set(unique_cities))
unique_cities.sort()
print(len(unique_cities), ' cities')
print(unique_cities)

84  cities
['amsterdam', 'antwerp', 'asheville', 'athens', 'austin', 'barcelona', 'barossa-valley', 'barwon-south-west-vic', 'beijing', 'bergamo', 'berlin', 'bologna', 'bordeaux', 'boston', 'bristol', 'brussels', 'cambridge', 'cape-town', 'chicago', 'clark-county-nv', 'columbus', 'copenhagen', 'denver', 'dublin', 'edinburgh', 'euskadi', 'florence', 'geneva', 'ghent', 'girona', 'greater-manchester', 'hawaii', 'hong-kong', 'istanbul', 'lisbon', 'london', 'los-angeles', 'lyon', 'madrid', 'malaga', 'mallorca', 'manchester', 'melbourne', 'menorca', 'milan', 'montreal', 'naples', 'nashville', 'new-orleans', 'new-york-city', 'northern-rivers', 'oakland', 'oslo', 'pacific-grove', 'paris', 'portland', 'porto', 'prague', 'puglia', 'quebec-city', 'rhode-island', 'rio-de-janeiro', 'rome', 'salem-or', 'san-diego', 'san-francisco', 'santa-clara-county', 'santa-cruz-county', 'seattle', 'sevilla', 'sicily', 'stockholm', 'sydney', 'taipei', 'tasmania', 'toronto', 'trentino', 'twin-cities-msa', 'vancouv

In [10]:
# run function on the list of cities 
start_time = time.time() # timestamp

for city in unique_cities:
    # if both files haven't been created, continue to create the consolidated csv files for that city
    if(not os.path.isfile(target + city + '_listings.csv') or not os.path.isfile(target + city + '_reviews.csv')):
        consolidate_data(city, directory, target)

time_to_run = (time.time() - start_time)/60 # timestamp, calculate function time
print('Time:',time_to_run, 'minutes')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1167: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (43,87) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DtypeWarning: Columns (43,61,62) have mix

Time: 186.9280581355095 minutes
